In [ ]:
import kagglehub

path = kagglehub.dataset_download("aryashah2k/mango-leaf-disease-dataset")

print("Path to dataset files:", path)

100%|██████████| 103M/103M [00:04<00:00, 26.8MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/aryashah2k/mango-leaf-disease-dataset/versions/1


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
import gc

POTATO_WEIGHTS_PATH = 'efficientnet_stage2_potato_finetuned.weights.h5'
FINAL_MODEL_SAVE_PATH = 'efficientnet_final_mango.weights.h5' 
FINAL_FINE_TUNE_LR = 5e-6 

MANGO_ROOT_PATH = "/root/.cache/kagglehub/datasets/aryashah2k/mango-leaf-disease-dataset/versions/1" # Corrected path

N_Mango = 8 

IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32

def load_and_configure_dataset(root_path, subset_name, validation_split_ratio):
    if not os.path.exists(root_path):
        raise FileNotFoundError(f"Dataset root path not found: {root_path}")

    
    dataset = tf.keras.utils.image_dataset_from_directory(
        root_path,
        labels='inferred',
        label_mode='categorical', 
        image_size=(IMG_HEIGHT, IMG_WIDTH),
        interpolation='bilinear',
        batch_size=BATCH_SIZE,
        shuffle=True if subset_name == 'training' else False, 
        seed=42, 
        validation_split=validation_split_ratio,
        subset=subset_name, 
    )

    def preprocess_image(image, label):
        image = tf.cast(image, tf.float32) / 255.0
        return image, label

    dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)

    if subset_name == 'training':
        data_augmentation = tf.keras.Sequential([
            tf.keras.layers.RandomFlip("horizontal_and_vertical"),
            tf.keras.layers.RandomRotation(0.2),
            tf.keras.layers.RandomZoom(0.2),
            tf.keras.layers.RandomContrast(0.2),
        ])
        dataset = dataset.map(lambda x, y: (data_augmentation(x, training=True), y),
                              num_parallel_calls=tf.data.AUTOTUNE)

    dataset = dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

mango_train_generator = load_and_configure_dataset(MANGO_ROOT_PATH, 'training', 0.2)
mango_validation_generator = load_and_configure_dataset(MANGO_ROOT_PATH, 'validation', 0.2)

Found 4000 files belonging to 8 classes.
Using 3200 files for training.
Found 4000 files belonging to 8 classes.
Using 800 files for validation.


In [ ]:
from tensorflow.keras.applications import EfficientNetB0

base_model = EfficientNetB0(weights=None, include_top=False, input_shape=(224, 224, 3))

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)
final_output = Dense(N_Mango, activation='softmax', name='mango_output')(x)

model_mango = Model(inputs=base_model.input, outputs=final_output)

model_mango.load_weights(POTATO_WEIGHTS_PATH, skip_mismatch=True)

for layer in model_mango.layers:
    layer.trainable = True

print(f"Model successfully adapted and loaded for {N_Mango} Mango classes.")

Model successfully adapted and loaded for 8 Mango classes.


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:648: UserWarning: A total of 1 objects could not be loaded. Example error message for object <Dense name=mango_output, built=True>:

The shape of the target variable and the shape of the target value in `variable.assign(value)` must match. variable.shape=(1280, 8), Received: value.shape=(1280, 3). Target variable: <Variable path=mango_output/kernel, shape=(1280, 8), dtype=float32, value=[[ 0.02899036 -0.02333876 -0.01758982 ...  0.0510546   0.01834152
   0.04277759]
 [-0.05509358  0.0229196   0.0250318  ...  0.00964396  0.06692811
  -0.01844483]
 [-0.01769584  0.04200593  0.02280359 ...  0.03884949 -0.04060157
   0.01213431]
 ...
 [ 0.03785417  0.02044968  0.04618819 ... -0.00055324  0.03685097
   0.03994401]
 [-0.06037849 -0.05936372  0.01806995 ...  0.0588972   0.03211098
   0.00956745]
 [ 0.03735261 -0.06418902 -0.06010298 ...  0.05420771  0.01113424
   0.01228601]]>

List of objects that could not be loaded:
[<D

In [ ]:
model_mango.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=FINAL_FINE_TUNE_LR),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

mango_callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath=FINAL_MODEL_SAVE_PATH,
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        save_weights_only=True,
        verbose=1
    ),
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, mode='max', verbose=1)
]

print("\n--- Starting Phase 2: Final Fine-Tuning on Mango Diseases ---")

history_mango = model_mango.fit(
    mango_train_generator,
    validation_data=mango_validation_generator,
    epochs=15, 
    callbacks=mango_callbacks
)



--- Starting Phase 2: Final Fine-Tuning on Mango Diseases ---
Epoch 1/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 10s/step - accuracy: 0.1622 - loss: 2.1087
Epoch 1: val_accuracy improved from -inf to 0.55000, saving model to efficientnet_final_mango.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 1070s 10s/step - accuracy: 0.1625 - loss: 2.1081 - val_accuracy: 0.5500 - val_loss: 1.9796
Epoch 2/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - accuracy: 0.2937 - loss: 1.8798
Epoch 2: val_accuracy did not improve from 0.55000
100/100 ━━━━━━━━━━━━━━━━━━━━ 977s 10s/step - accuracy: 0.2942 - loss: 1.8792 - val_accuracy: 0.2500 - val_loss: 1.9565
Epoch 3/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - accuracy: 0.4358 - loss: 1.6751
Epoch 3: val_accuracy did not improve from 0.55000
100/100 ━━━━━━━━━━━━━━━━━━━━ 972s 10s/step - accuracy: 0.4362 - loss: 1.6746 - val_accuracy: 0.0600 - val_loss: 2.1730
Epoch 4/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - accuracy: 0.5482 - loss: 1.4765
Epoch 4: val_accuracy did not i